In [1]:
import json
import jinja2
import nestedtext as nt
import os
import re
import openpyxl
AV_bench_env = jinja2.Environment(loader=jinja2.FileSystemLoader(searchpath='/projects/H3A0/workspaces/jdavid/h3v1_release/UVM_tools/uvm_analog_tb/AV_bench/templates/'))
print ("hello world")

hello world


In [2]:
dut_ports = nt.load("simple_uvm_testcase_pins.nt",top='dict')
dut_spec = {'name': dut_ports['cellname'],'library': dut_ports['lib'], 'ports':{}}
error_count = 0
for port in dut_ports['ports']:
    match_obj = re.search(r'(?P<name>\w[_\w\d]*)((?:<(?P<bit>\d+)>)|(?:<(?P<msb>\d+):(?P<lsb>\d+)>))',port)
    if match_obj: 
        if match_obj.group('bit'):
            error_count += 1
            print( "Illegal split bus ",port, '\tusing:',f'{match_obj.group("name")}_{match_obj.group("bit")}')
            dut_spec['ports'][f'{match_obj.group("name")}_{match_obj.group("bit")}'] = {'direction':dut_ports['ports'][port]}
        if match_obj.group("msb"):
            dut_spec['ports'][match_obj.group("name")] = {'direction':dut_ports['ports'][port],'msb':match_obj.group('msb'),'lsb':match_obj.group('lsb') }
            #print("range found", port ,match_obj.group("name"),match_obj.group('msb'))
    else:
        dut_spec['ports'][port] = {'direction':dut_ports['ports'][port]}
            
            
if error_count:
    print(error_count, 'Errors - suggest fixing the design to remove them')


In [3]:

print( json.dumps(dut_spec, indent=4))

{
    "name": "simple_uvm_testcase",
    "library": "RFDV_scratch",
    "ports": {
        "AVDD1P8": {
            "direction": "input"
        },
        "AVSS": {
            "direction": "input"
        },
        "bg_enable": {
            "direction": "input"
        },
        "clk_en": {
            "direction": "input",
            "msb": "3",
            "lsb": "0"
        },
        "mclk_in": {
            "direction": "input"
        },
        "mode_sel_3": {
            "direction": "input",
            "msb": "1",
            "lsb": "0"
        },
        "mode_sel_2": {
            "direction": "input",
            "msb": "1",
            "lsb": "0"
        },
        "mode_sel_1": {
            "direction": "input",
            "msb": "1",
            "lsb": "0"
        },
        "mode_sel_0": {
            "direction": "input",
            "msb": "1",
            "lsb": "0"
        },
        "clk_out_3": {
            "direction": "output"
        },
        "clk_o

In [4]:
if "name" in [*dut_spec]:
    print([dut_spec['name']])
    tb_name = '_'.join([dut_spec['name'],'tb'])
    print(tb_name)
try: 
    ports = dut_spec['ports']
except:
    print( "ports not in dut")
#print(ports) 
if "library" in [*dut_spec] :
    dut_library = dut_spec['library']
    tb_library = '_'.join([dut_spec['library'],'sim'])
print (tb_library)



['simple_uvm_testcase']
simple_uvm_testcase_tb
RFDV_scratch_sim


In [5]:
# nameconvention = { 
#     'groups':{
#         "register": {
#             "patterns": [
#                 "digi",
#                 "digo"
#             ],
#             'after': '_',
#             'before': '$',
#             'priority': 0,
#             'porttype': 'register'                
#         },
#         'controls': {
#             'after': '_',
#             "patterns": [
#                 'en','sel','fc','byp','ctrl','reset','rst','adj','set','vset','pol','inv'
#             ],
#             'priority': 7,
#             'porttype': 'register'                
#         },
#         "power": {
#             "patterns": [
#                 "vdd",
#                 "VDD"
#             ],
#             'priority': 2,
#             'porttype': 'power',
#             'implies': {'supply_type':"volts"}                
#         },
#         "ground": {
#             "patterns": [
#                 "vss",
#                 "VSS",
#                 "gnd",
#                 "GND"
#             ],
#             'priority': 3,
#             'porttype': 'power',
#             'implies': {'supply_type':"ground"}                
#         },
#         "vldo": {
#             "patterns": [
#                 "ldo"
#             ],
#             'implies': {'supply_type':"volts"},
#             'priority': 4,
#             'porttype': 'power'                
#         },
#         'resets': {
#             'after': '_',
#             'patterns': [
#                 'reset','rst'
#             ],
#             'priority': 5,
#             'porttype': 'digital'  
#         },
#         'clock': {
#             'after': '_',
#             'patterns': ['clk','CLK','ck','sync','LO'],
#             'priority': 6,
#             'porttype': 'digital'
#         },
#         'data': {
#             'after':'_',
#             'patterns':['data','dta','dith','HS_TEST'],
#             'priority': 1,
#             'porttype': 'digital'
#         },
#         'sdata': {
#             'patterns': ['IL[0-9][AB]_[RT]X'],
#             'priority': 8,
#             'porttype': 'digital'
#         },
#         'ibias': {
#             'patterns': [
#                 'ipt',
#                 'IPT',
#                 'ibg',
#                 'IBG',
#                 'inp',
#                 'INP'
#             ],
#             'priority': 9,
#             'implies': {'bias_type':"amps"},                
#             'porttype': 'bias'  
#         }
#     },
#     'default': {
#         'porttype': 'analog'
#     },
#     'values':{
#         'ibias_nom': {
#             'patterns': {
#                 '\d*[pP]?\d+[uU]\d*'
#             },
#             'check_porttypes': ['bias']
#         },
#         'supply_nom':{
#             'patterns': {
#                 '\d+[pP]\d+'
#             },
#             'check_groups': ['supply']
#         }
#     }
# }
# print(nt.dumps(nameconvention)) 
# nt.dump(nameconvention, 'namingconvention.nt')


In [6]:
name_convention = nt.load("namingconvention.nt",top='dict')

In [7]:
groupsearches = []
value_search = {}
for group in name_convention['groups']:
    group_patterns = '|'.join(name_convention['groups'][group]['patterns'])
    searchgroup = "(?P<{0}>{1}(?:{2}){3})".format(
        group, 
        name_convention['groups'][group].get('after', ''),
        group_patterns,
        name_convention['groups'][group].get('before', '')
        )
    groupsearches.append(  searchgroup )
group_search = re.compile( r'|'.join(groupsearches))
for value in name_convention['values']:
    value_patterns = '|'.join(name_convention['values'][value]['patterns'])
    searchvalues = "(?P<{0}>{1}(?:{2}){3})".format(
        value, 
        name_convention['values'][value].get('after', ''),
        value_patterns,
        name_convention['values'][value].get('before', '')
        )
    value_search[value] = re.compile(searchvalues)
print (value_search)


{'ibias_nom': re.compile('(?P<ibias_nom>(?:\\d*[pP]?\\d+[uU]\\d*))'), 'supply_nom': re.compile('(?P<supply_nom>(?:\\d+[pP]\\d+))')}


In [8]:
for port in ports:
    porttypes = []
    implied ={}
    search_result = re.finditer(group_search,port)
    for match_object in search_result:
        groups = match_object.groupdict()
        for group in groups.keys():
            if match_object.group(group):
                porttypes.append(group)
    porttype = ''
    for group in sorted(name_convention['groups'], key=lambda group: name_convention['groups'][group]['priority'] ):
        #print (group, name_convention['groups'][group]['priority'])
        if group in porttypes:
            porttype = name_convention['groups'][group]['porttype']
            implied = name_convention["groups"][group].get('implies', {})
            for value in name_convention['values']:
                if group in name_convention['values'][value].get('check_groups',[]):
                    value_result = re.search(value_search[value],port)
                    if value_result:
                        dut_spec['ports'][port][value] = value_result.group(value).lower().replace('p','.',1).replace('u','e-6')
            break
    else: 
        porttype = name_convention['default']['porttype']
       
    # for group in name_convention['groups']:
    #     if group in porttypes:
    #         porttype = name_convention['groups'][group]['porttype']
    #         implied = name_convention["groups"][group].get('implies', {})
    #         for value in name_convention['values']:
    #             if group in name_convention['values'][value].get('check_groups',[]):
    #                 value_result = re.search(value_search[value],port)
    #                 dut_spec['ports'][port][value] = value_result.group(value).lower().replace('p','.',1).replace('u','e-6')
    #         break
    # else: 
    #     porttype = name_convention['default']['porttype']
    dut_spec['ports'][port]['porttype'] = porttype
    for key,value in implied.items():
        dut_spec['ports'][port][key] = value;
    for value in name_convention['values']:
        if porttype in name_convention['values'][value].get('check_porttypes',[]):
            value_result = re.search(value_search[value],port)
            if value_result and value_result.group(value):
                dut_spec['ports'][port][value] = value_result.group(value).lower().replace('p','.',1).replace('u','e-6')

            
print(nt.dumps(dut_spec,sort_keys=True))  

library: RFDV_scratch
name: simple_uvm_testcase
ports:
    AVDD1P8:
        direction: input
        porttype: power
        supply_nom: 1.8
        supply_type: volts
    AVSS:
        direction: input
        porttype: power
        supply_type: ground
    bg_enable:
        direction: input
        porttype: register
    clk_en:
        direction: input
        lsb: 0
        msb: 3
        porttype: register
    clk_out_0:
        direction: output
        porttype: digital
    clk_out_1:
        direction: output
        porttype: digital
    clk_out_2:
        direction: output
        porttype: digital
    clk_out_3:
        direction: output
        porttype: digital
    dll_lock_status:
        direction: output
        lsb: 0
        msb: 3
        porttype: register
    mclk_in:
        direction: input
        porttype: digital
    mode_sel_0:
        direction: input
        lsb: 0
        msb: 1
        porttype: register
    mode_sel_1:
        direction: input
        l

In [9]:

nt.dump(dut_spec, "dut_spec.nt",sort_keys=True)
